In [1]:
import os
os.chdir("/RAFT-Stereo/RGBNIR") # Fix with forder root path

In [4]:
from arguments import Arguments
from net.feature_fusion import RAFTStereoFusion
import torch
args = Arguments()
args.shared_fusion = True
args.shared_backbone = False
args.corr_implementation = "reg"
args.n_gru_layers = 3
args.n_downsample = 2
model = RAFTStereoFusion(args).cuda()
model.load_state_dict(torch.load("weights/model_feature_fusion.pth")["model_state_dict"])
model.eval()



RAFTStereoFusion(
  (update_block): BasicMultiUpdateBlock(
    (encoder): BasicMotionEncoder(
      (convc1): Conv2d(36, 64, kernel_size=(1, 1), stride=(1, 1))
      (convc2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convf1): Conv2d(2, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (convf2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv): Conv2d(128, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (gru08): ConvGRU(
      (convz): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convr): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convq): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (gru16): ConvGRU(
      (convz): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convr): Conv2d(384, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convq): Conv2d(384, 128, kernel_siz

In [ ]:
import numpy as np
from visualize.vis_utils.widget import FrameExplorer
from visualize.vis_utils.inputs import get_valid_input_from_path
import matplotlib.pyplot as plt


def plot_dataset(path: str):
    inputs = get_valid_input_from_path(path)
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(inputs[0][0].permute(1, 2, 0).numpy().astype(np.uint8))
    plt.subplot(1, 4, 2)
    plt.imshow(inputs[2][0].permute(1, 2, 0).numpy().astype(np.uint8), cmap="gray")
    plt.subplot(1, 4, 3)
    plt.imshow(np.zeros((540, 720), np.uint8), cmap="gray")
    plt.scatter(
        inputs[4][..., 0],
        inputs[4][..., 1],
        c=inputs[4][..., 2],
        s=1,
        cmap="magma",
        vmin=0,
        vmax=32,
    )
    plt.xlim(0, 720)
    with torch.no_grad():
        flow = -model(*[x.cuda() for x in inputs[:4]], test_mode=True)[1]
    plt.subplot(1,4,4)
    plt.imshow(flow.cpu()[0,0], vmin=0, vmax=32, cmap="magma")
    plt.show()

FrameExplorer(plot_dataset)